# Offline Assignment

This is an offline assigment meant for us in Commit to understand how you approach data science problems, and how you implement said approach.


It is 100% OK if you don't succeed finishing some of the sub-tasks- in fact, it is typical not get everything right.

We wanted to give you the best chance to show us what you bring to the table no matter which field of data science interests you most.


We have many tasks in Commit that are very similar to the task you see here.


**Important note- when you encounter a problem, all approaches that make sense are valid. Tradeoffs are the nature of the field, and it is likely that your considerations for tradeoffs will somewhat vary from those of the interviewer. As long as you can explain your considerations (or most of them), this is 100% fine**

# Use Case - NLP


## Description

You are given a task in the field of NLP.
Our client, a large scientific corporation, is interested in using LLMs to improve its scientific work.

## Our First Task

Part of the scientific work involves making internet search for a given topic, and presenting only the important material to a researcher.


For that, they would like you to mine a portion of the web (several web pages are enough) and create a vector database that you can query.


Please mine several web pages (from ArXiv or Pubmed), extract the text from them, then split the text in a meaningful way, then build a vector database that one can query to retrieve the needed data.


## Our Follow-up Task

Now that you are done, the client wants more (clients almost always do).

The client now wants to be able to ask complex questions (like “what are the dangerous medical conditions oh wise chatbot”) and still get an answer based on their data.


Upon thinking, you came up with a solution- you will break the question into sub steps, query the vector database about each sub-step, and then compose an answer.

Please create a flow that allows a person to ask complex questions and still get a reply based on their data. This flow should break down the question to the right substeps and then compose a good answer.  


## Our Next Follow-up Task

Now that you are done, the client is satisfied and wants to move from POC into production.

Please harden your code and add as much of the items needed for code to be production-grade as you can.

Examples are: log mockups, error handling, etc.

Remember - the key here is to see that you know what to add and what to look for, implementation is secondary

## Next Follow Up Task - Evaluation

Now, the customer is pleased but they raised a question- how do I know that my model produces accurate results?

Please think of a way to measure how accurate the model is.

Please make sure that the measuring method actually measures the quality of the answer.

## Task 1: Web Mining & Vector Database

## Architecture Diagram

```
┌─────────────────────────────────────────────────────────────────┐
│                        DATA INGESTION                          │
│                                                                 │
│  ┌──────────┐    ┌──────────────┐    ┌────────────────┐        │
│  │  ArXiv /  │───▶│  Web Scraper  │───▶│ Text Extractor │        │
│  │  PubMed   │    │ (httpx async  │    │ (BeautifulSoup)│        │
│  │  URLs     │    │  + rate limit)│    └───────┬────────┘        │
│  └──────────┘    └──────────────┘            │                  │
│                                               ▼                  │
│                                    ┌──────────────────┐         │
│                                    │  Text Chunker     │         │
│                                    │  (RecursiveChar   │         │
│                                    │   TextSplitter)   │         │
│                                    └────────┬─────────┘         │
│                                             ▼                    │
│                                    ┌──────────────────┐         │
│                                    │  Embedding Model  │         │
│                                    │  (sentence-       │         │
│                                    │   transformers)   │         │
│                                    └────────┬─────────┘         │
│                                             ▼                    │
│                                    ┌──────────────────┐         │
│                                    │  ChromaDB Vector  │         │
│                                    │  Store            │         │
│                                    └──────────────────┘         │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│                      QUERY PIPELINE                             │
│                                                                 │
│  ┌──────────┐    ┌──────────────┐    ┌────────────────┐        │
│  │  User     │───▶│ Query        │───▶│ Question        │        │
│  │  Question │    │ Decomposer   │    │ Decomposition   │        │
│  └──────────┘    │ (LLM)        │    │ (Sub-queries)   │        │
│                  └──────────────┘    └───────┬────────┘         │
│                                              ▼                   │
│                                    ┌─────────────────┐          │
│                                    │ Vector DB Query  │          │
│                                    │ (per sub-query)  │          │
│                                    └────────┬────────┘          │
│                                             ▼                    │
│                                    ┌─────────────────┐          │
│                                    │ Answer Composer  │          │
│                                    │ (LLM synthesis)  │          │
│                                    └────────┬────────┘          │
│                                             ▼                    │
│                                    ┌─────────────────┐          │
│                                    │  Final Answer    │          │
│                                    └─────────────────┘          │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# your solution here
# Install dependencies
!pip install -q httpx beautifulsoup4 chromadb sentence-transformers langchain langchain-text-splitters openai tiktoken

In [ ]:
import httpx
import asyncio
from bs4 import BeautifulSoup
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time
import re
from typing import List, Dict, Optional
import hashlib

### Step 1: Web Mining — Scrape ArXiv & PubMed Pages

In [ ]:
# Define target papers
# Topic: Medical AI / dangerous medical conditions (aligns with Task 2's example query)
ARXIV_URLS = [
    "https://arxiv.org/abs/2308.14089",  # Large Language Models in Medicine
    "https://arxiv.org/abs/2305.09617",  # Clinical LLM applications
    "https://arxiv.org/abs/2307.14334",  # Med-PaLM
    "https://arxiv.org/abs/2212.13138",  # ChatGPT in healthcare
    "https://arxiv.org/abs/2306.05052",  # Biomedical NLP
]

PUBMED_URLS = [
    "https://pubmed.ncbi.nlm.nih.gov/37548974/",
    "https://pubmed.ncbi.nlm.nih.gov/36930957/",
]

HEADERS = {"User-Agent": "Mozilla/5.0 (Research Bot; educational purposes)"}


async def scrape_arxiv_page(client: httpx.AsyncClient, url: str) -> Dict[str, str]:
    """Scrape an ArXiv abstract page and extract title + abstract text."""
    response = await client.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    title_tag = soup.find("h1", class_="title")
    title = title_tag.get_text(strip=True).replace("Title:", "").strip() if title_tag else "Unknown Title"

    abstract_tag = soup.find("blockquote", class_="abstract")
    abstract = abstract_tag.get_text(strip=True).replace("Abstract:", "").strip() if abstract_tag else ""

    # Try to get full paper text from HTML version
    html_url = url.replace("/abs/", "/html/")
    full_text = ""
    try:
        html_response = await client.get(html_url)
        if html_response.status_code == 200:
            html_soup = BeautifulSoup(html_response.text, "html.parser")
            article = html_soup.find("article") or html_soup.find("div", class_="ltx_page_content")
            if article:
                for ref in article.find_all(["section"], class_=lambda x: x and "bib" in str(x).lower()):
                    ref.decompose()
                full_text = article.get_text(separator=" ", strip=True)
    except Exception:
        pass

    content = full_text if full_text else abstract
    return {"title": title, "url": url, "content": content, "source": "arxiv"}


async def scrape_pubmed_page(client: httpx.AsyncClient, url: str) -> Dict[str, str]:
    """Scrape a PubMed page for title and abstract."""
    response = await client.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    title_tag = soup.find("h1", class_="heading-title")
    title = title_tag.get_text(strip=True) if title_tag else "Unknown Title"

    abstract_tag = soup.find("div", class_="abstract-content")
    abstract = abstract_tag.get_text(strip=True) if abstract_tag else ""

    return {"title": title, "url": url, "content": abstract, "source": "pubmed"}


async def scrape_all_pages() -> List[Dict[str, str]]:
    """Scrape all pages concurrently with semaphore-based rate limiting."""
    semaphore = asyncio.Semaphore(3)  # Max 3 concurrent requests

    async def _scrape_with_limit(coro, url: str, source: str):
        async with semaphore:
            try:
                doc = await coro
                print(f"  ✓ [{source}] {doc['title'][:60]}... ({len(doc['content'])} chars)")
                return doc
            except Exception as e:
                print(f"  ✗ Failed to scrape {url}: {e}")
                return None

    async with httpx.AsyncClient(
        headers=HEADERS,
        follow_redirects=True,
        timeout=httpx.Timeout(15.0),
    ) as client:
        tasks = []
        print("Scraping ArXiv & PubMed pages...")
        for url in ARXIV_URLS:
            tasks.append(_scrape_with_limit(scrape_arxiv_page(client, url), url, "arxiv"))
        for url in PUBMED_URLS:
            tasks.append(_scrape_with_limit(scrape_pubmed_page(client, url), url, "pubmed"))

        results = await asyncio.gather(*tasks)
        documents = [doc for doc in results if doc is not None]

    print(f"\nTotal documents scraped: {len(documents)}")
    return documents


# In Jupyter/Colab, the event loop is already running — use await directly
documents = await scrape_all_pages()

### Step 2: Text Splitting / Chunking

**Design Decisions:**
- `RecursiveCharacterTextSplitter` splits on natural boundaries (paragraphs → sentences → words)
- Chunk size of 500 chars with 100 char overlap — good balance for scientific text
- Each chunk retains metadata (source URL, title) for traceability

In [ ]:
def chunk_documents(documents: List[Dict], chunk_size: int = 500, chunk_overlap: int = 100) -> List[Dict]:
    """Split documents into meaningful chunks with metadata."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
        length_function=len,
    )

    all_chunks = []
    for doc in documents:
        text = re.sub(r'\s+', ' ', doc["content"]).strip()
        if not text:
            continue

        chunks = splitter.split_text(text)
        for i, chunk in enumerate(chunks):
            chunk_id = hashlib.md5(f"{doc['url']}_{i}".encode()).hexdigest()
            all_chunks.append({
                "id": chunk_id,
                "text": chunk,
                "metadata": {
                    "source_url": doc["url"],
                    "title": doc["title"],
                    "source": doc["source"],
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                }
            })

    return all_chunks


chunks = chunk_documents(documents)
print(f"Total chunks created: {len(chunks)}")
print(f"\nSample chunk:")
print(f"  ID: {chunks[0]['id']}")
print(f"  Text: {chunks[0]['text'][:200]}...")
print(f"  Metadata: {chunks[0]['metadata']}")

### Step 3: Build Vector Database with ChromaDB

In [ ]:
# Initialize embedding model (all-MiniLM-L6-v2: fast + good quality)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB (in-memory for POC)
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(
    name="scientific_papers",
    metadata={"hnsw:space": "cosine"}
)

# Embed and insert chunks in batches
print("Embedding and inserting chunks...")
batch_size = 50
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i + batch_size]
    texts = [c["text"] for c in batch]
    ids = [c["id"] for c in batch]
    metadatas = [c["metadata"] for c in batch]
    embeddings = embedding_model.encode(texts).tolist()
    collection.add(documents=texts, embeddings=embeddings, metadatas=metadatas, ids=ids)
    print(f"  Inserted batch {i // batch_size + 1} ({len(batch)} chunks)")

print(f"\nVector database ready. Total documents: {collection.count()}")

In [ ]:
def query_vector_db(query: str, n_results: int = 3) -> List[Dict]:
    """Query the vector database and return relevant chunks."""
    query_embedding = embedding_model.encode([query]).tolist()
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return [
        {
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i],
            "distance": results["distances"][0][i],
        }
        for i in range(len(results["ids"][0]))
    ]


# Test query
test_results = query_vector_db("What are the risks of using AI in medical diagnosis?")
print("Query: 'What are the risks of using AI in medical diagnosis?'\n")
for i, r in enumerate(test_results):
    print(f"Result {i+1} (distance: {r['distance']:.4f}):")
    print(f"  Source: {r['metadata']['title'][:60]}")
    print(f"  Text: {r['text'][:200]}...\n")

---
## Task 2: Complex Question Answering via Query Decomposition

### Approach: Multi-Step RAG with Query Decomposition

1. **Decompose** the complex question into atomic sub-questions using an LLM
2. **Retrieve** relevant chunks for each sub-question from the vector DB
3. **Synthesize** a final answer using all retrieved context + the original question

In [ ]:
import os
import json
from openai import OpenAI

# Set your API key (use env vars or secrets manager in production)
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your key
llm_client = OpenAI()


def decompose_question(question: str) -> List[str]:
    """Break a complex question into simpler, searchable sub-questions via LLM."""
    prompt = f"""You are a research assistant. Given a complex question, break it down into
2-5 simpler, specific sub-questions that can each be answered independently by searching
a scientific literature database.

Rules:
- Each sub-question should be self-contained and searchable
- Cover all aspects of the original question
- Return ONLY a JSON list of strings, no other text

Complex question: "{question}"

Sub-questions (JSON list):"""

    response = llm_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=500,
    )

    raw = response.choices[0].message.content.strip()
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        match = re.search(r'\[.*\]', raw, re.DOTALL)
        if match:
            return json.loads(match.group())
        return [question]


def retrieve_for_subquestions(sub_questions: List[str], n_results_per_query: int = 3) -> List[Dict]:
    """Retrieve relevant chunks for each sub-question, deduplicated."""
    all_results = []
    seen_ids = set()
    for sq in sub_questions:
        results = query_vector_db(sq, n_results=n_results_per_query)
        for r in results:
            chunk_id = r["metadata"]["source_url"] + "_" + str(r["metadata"]["chunk_index"])
            if chunk_id not in seen_ids:
                seen_ids.add(chunk_id)
                r["sub_question"] = sq
                all_results.append(r)
    return all_results


def synthesize_answer(question: str, sub_questions: List[str], context_chunks: List[Dict]) -> str:
    """Synthesize a final answer from retrieved context using LLM."""
    context_parts = [
        f"[Source {i+1}: {chunk['metadata']['title']}]\n{chunk['text']}"
        for i, chunk in enumerate(context_chunks)
    ]
    context_str = "\n\n".join(context_parts)

    prompt = f"""You are a scientific research assistant. Answer the user's question based ONLY
on the provided context from scientific papers. If the context doesn't contain enough information
to fully answer, say so explicitly.

The question was broken into these sub-questions for research:
{json.dumps(sub_questions, indent=2)}

Retrieved Context:
{context_str}

Original Question: {question}

Please provide a comprehensive answer with citations to the source papers where applicable.
Answer:"""

    response = llm_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=1000,
    )
    return response.choices[0].message.content.strip()


def ask_complex_question(question: str) -> Dict:
    """End-to-end pipeline: decompose → retrieve → synthesize."""
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print(f"{'='*60}")

    print("\n📋 Step 1: Decomposing question...")
    sub_questions = decompose_question(question)
    for i, sq in enumerate(sub_questions):
        print(f"  Sub-Q{i+1}: {sq}")

    print("\n🔍 Step 2: Retrieving relevant context...")
    context_chunks = retrieve_for_subquestions(sub_questions)
    print(f"  Retrieved {len(context_chunks)} unique chunks")

    print("\n🤖 Step 3: Synthesizing answer...")
    answer = synthesize_answer(question, sub_questions, context_chunks)
    print(f"\n📝 Answer:\n{answer}")

    return {
        "question": question,
        "sub_questions": sub_questions,
        "context_chunks": context_chunks,
        "answer": answer,
    }

In [ ]:
# Test with the example complex question
result = ask_complex_question(
    "What are the dangerous medical conditions that AI systems might misdiagnose, "
    "and what safety measures should be put in place?"
)

In [ ]:
# Test with another complex question
result2 = ask_complex_question(
    "How do large language models compare to traditional NLP methods in "
    "biomedical text mining, and what are the limitations of each approach?"
)

---
## Task 3: Production-Hardened Code

### What production-grade code needs:
1. **Structured logging** with levels (DEBUG, INFO, WARNING, ERROR)
2. **Error handling** with retries and graceful degradation
3. **Configuration management** (no hardcoded values)
4. **Input validation and sanitization**
5. **Rate limiting and circuit breakers**
6. **Health checks and monitoring hooks**
7. **Type hints and documentation**
8. **Idempotent operations** (safe to retry)
9. **Metrics collection** for observability
10. **Graceful shutdown handling**

In [ ]:
import logging
import time
import hashlib
import json
import re
import os
import asyncio
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any, Tuple
from functools import wraps
from datetime import datetime

import httpx
from bs4 import BeautifulSoup
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI


# ============================================================
# 1. CONFIGURATION
# ============================================================

@dataclass
class AppConfig:
    """Centralized configuration. In production, load from env vars / config file."""
    # Embedding
    embedding_model_name: str = "all-MiniLM-L6-v2"
    chunk_size: int = 500
    chunk_overlap: int = 100

    # Vector DB
    collection_name: str = "scientific_papers"
    chroma_persist_dir: str = "./chroma_db"
    similarity_metric: str = "cosine"
    default_n_results: int = 3

    # Scraping (httpx)
    request_timeout: float = 15.0
    max_concurrent_requests: int = 3
    max_retries: int = 3
    max_connections: int = 10
    max_keepalive_connections: int = 5

    # LLM
    llm_model: str = "gpt-3.5-turbo"
    llm_temperature_decompose: float = 0.0
    llm_temperature_synthesize: float = 0.2
    llm_max_tokens: int = 1000
    max_context_chunks: int = 15

    # Monitoring
    log_level: str = "INFO"


# ============================================================
# 2. LOGGING SETUP
# ============================================================

def setup_logging(config: AppConfig) -> logging.Logger:
    """Configure structured logging."""
    logger = logging.getLogger("rag_pipeline")
    logger.setLevel(getattr(logging, config.log_level))
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter(
            "%(asctime)s | %(levelname)-8s | %(name)s | %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    # In production: add RotatingFileHandler, JSON formatter for ELK/Datadog, Sentry
    return logger


# ============================================================
# 3. METRICS COLLECTOR
# ============================================================

@dataclass
class PipelineMetrics:
    """Collects metrics for monitoring/alerting. In production, push to Prometheus/Datadog."""
    scrape_success: int = 0
    scrape_failures: int = 0
    chunks_created: int = 0
    queries_processed: int = 0
    avg_retrieval_time_ms: float = 0.0
    avg_llm_latency_ms: float = 0.0
    total_tokens_used: int = 0
    errors: List[Dict] = field(default_factory=list)

    def record_error(self, component: str, error: str):
        self.errors.append({
            "timestamp": datetime.utcnow().isoformat(),
            "component": component,
            "error": error,
        })

    def to_dict(self) -> Dict:
        return {
            "scrape_success": self.scrape_success,
            "scrape_failures": self.scrape_failures,
            "chunks_created": self.chunks_created,
            "queries_processed": self.queries_processed,
            "total_tokens_used": self.total_tokens_used,
            "recent_errors": self.errors[-10:],
        }


# ============================================================
# 4. INPUT VALIDATION
# ============================================================

class InputValidator:
    """Validates and sanitizes user inputs."""
    MAX_QUERY_LENGTH = 2000
    ALLOWED_URL_PATTERNS = [
        r"^https?://arxiv\.org/",
        r"^https?://pubmed\.ncbi\.nlm\.nih\.gov/",
    ]

    @staticmethod
    def validate_query(query: str) -> str:
        if not query or not query.strip():
            raise ValueError("Query cannot be empty")
        query = query.strip()
        if len(query) > InputValidator.MAX_QUERY_LENGTH:
            raise ValueError(f"Query exceeds max length of {InputValidator.MAX_QUERY_LENGTH}")
        query = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f]', '', query)
        return query

    @staticmethod
    def validate_url(url: str) -> bool:
        return any(re.match(p, url) for p in InputValidator.ALLOWED_URL_PATTERNS)


# ============================================================
# 5. PRODUCTION RAG PIPELINE
# ============================================================

class ProductionRAGPipeline:
    """
    Production-grade RAG pipeline for scientific literature search.

    Features:
    - Async HTTP via httpx with connection pooling & transport-level retries
    - Structured logging at every stage
    - Input validation and sanitization
    - Metrics collection for monitoring
    - Graceful error handling and degradation
    - Idempotent upserts (safe to re-run ingestion)
    - Configurable via AppConfig
    """

    def __init__(self, config: Optional[AppConfig] = None):
        self.config = config or AppConfig()
        self.logger = setup_logging(self.config)
        self.metrics = PipelineMetrics()
        self.validator = InputValidator()

        self.logger.info("Initializing RAG Pipeline...")

        try:
            self.embedding_model = SentenceTransformer(self.config.embedding_model_name)
            self.logger.info(f"Loaded embedding model: {self.config.embedding_model_name}")
        except Exception as e:
            self.logger.critical(f"Failed to load embedding model: {e}")
            raise

        try:
            self.chroma_client = chromadb.PersistentClient(path=self.config.chroma_persist_dir)
            self.collection = self.chroma_client.get_or_create_collection(
                name=self.config.collection_name,
                metadata={"hnsw:space": self.config.similarity_metric}
            )
            self.logger.info(f"ChromaDB ready. {self.collection.count()} documents")
        except Exception as e:
            self.logger.critical(f"Failed to initialize ChromaDB: {e}")
            raise

        self.llm_client = OpenAI()
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config.chunk_size,
            chunk_overlap=self.config.chunk_overlap,
            separators=["\n\n", "\n", ". ", " ", ""],
        )

        # httpx transport with built-in retries
        self._http_transport = httpx.AsyncHTTPTransport(retries=self.config.max_retries)
        self._http_limits = httpx.Limits(
            max_connections=self.config.max_connections,
            max_keepalive_connections=self.config.max_keepalive_connections,
        )

        self.logger.info("RAG Pipeline initialized successfully")

    def _get_http_client(self) -> httpx.AsyncClient:
        """Create a configured httpx async client. Use as async context manager."""
        return httpx.AsyncClient(
            transport=self._http_transport,
            limits=self._http_limits,
            timeout=httpx.Timeout(self.config.request_timeout),
            follow_redirects=True,
            headers={"User-Agent": "Mozilla/5.0 (Scientific RAG Bot)"},
        )

    # ---- INGESTION ----

    async def ingest_urls(self, urls: List[str]) -> Dict[str, int]:
        """Ingest multiple URLs concurrently with rate limiting. Returns {url: chunk_count}."""
        results = {}
        semaphore = asyncio.Semaphore(self.config.max_concurrent_requests)

        async with self._get_http_client() as client:
            async def _ingest_one(url: str):
                async with semaphore:
                    try:
                        n = await self._ingest_single_url(client, url)
                        results[url] = n
                    except Exception as e:
                        self.logger.error(f"Failed to ingest {url}: {e}")
                        results[url] = 0

            await asyncio.gather(*[_ingest_one(url) for url in urls])
        return results

    async def _ingest_single_url(self, client: httpx.AsyncClient, url: str) -> int:
        self.logger.info(f"Ingesting URL: {url}")
        if not self.validator.validate_url(url):
            raise ValueError(f"URL not from an allowed source: {url}")

        try:
            response = await client.get(url)
            response.raise_for_status()
            self.metrics.scrape_success += 1
        except httpx.HTTPStatusError as e:
            self.metrics.scrape_failures += 1
            self.metrics.record_error("scraper", f"HTTP {e.response.status_code}")
            raise
        except httpx.RequestError as e:
            self.metrics.scrape_failures += 1
            self.metrics.record_error("scraper", str(e))
            raise

        soup = BeautifulSoup(response.text, "html.parser")
        if "arxiv.org" in url:
            doc = self._parse_arxiv(soup, url)
        elif "pubmed" in url:
            doc = self._parse_pubmed(soup, url)
        else:
            raise ValueError(f"Unsupported source: {url}")

        if not doc["content"]:
            self.logger.warning(f"No content extracted from {url}")
            return 0

        chunks = self._create_chunks(doc)
        self._store_chunks(chunks)
        self.logger.info(f"Ingested {len(chunks)} chunks from {doc['title'][:50]}")
        return len(chunks)

    def _parse_arxiv(self, soup, url):
        title_tag = soup.find("h1", class_="title")
        title = title_tag.get_text(strip=True).replace("Title:", "").strip() if title_tag else "Unknown"
        abstract_tag = soup.find("blockquote", class_="abstract")
        content = abstract_tag.get_text(strip=True).replace("Abstract:", "").strip() if abstract_tag else ""
        return {"title": title, "url": url, "content": content, "source": "arxiv"}

    def _parse_pubmed(self, soup, url):
        title_tag = soup.find("h1", class_="heading-title")
        title = title_tag.get_text(strip=True) if title_tag else "Unknown"
        abstract_tag = soup.find("div", class_="abstract-content")
        content = abstract_tag.get_text(strip=True) if abstract_tag else ""
        return {"title": title, "url": url, "content": content, "source": "pubmed"}

    def _create_chunks(self, doc):
        text = re.sub(r'\s+', ' ', doc["content"]).strip()
        raw_chunks = self.text_splitter.split_text(text)
        chunks = []
        for i, chunk_text in enumerate(raw_chunks):
            chunk_id = hashlib.md5(f"{doc['url']}_{i}".encode()).hexdigest()
            chunks.append({
                "id": chunk_id,
                "text": chunk_text,
                "metadata": {
                    "source_url": doc["url"],
                    "title": doc["title"],
                    "source": doc["source"],
                    "chunk_index": i,
                    "total_chunks": len(raw_chunks),
                    "ingested_at": datetime.utcnow().isoformat(),
                }
            })
        self.metrics.chunks_created += len(chunks)
        return chunks

    def _store_chunks(self, chunks):
        """Embed and store. Uses upsert for idempotency."""
        texts = [c["text"] for c in chunks]
        ids = [c["id"] for c in chunks]
        metadatas = [c["metadata"] for c in chunks]
        embeddings = self.embedding_model.encode(texts).tolist()
        self.collection.upsert(documents=texts, embeddings=embeddings, metadatas=metadatas, ids=ids)

    # ---- QUERY ----

    def query(self, question: str) -> Dict[str, Any]:
        """Full RAG pipeline: validate → decompose → retrieve → synthesize."""
        start_time = time.time()
        self.logger.info(f"Processing query: {question[:100]}")
        question = self.validator.validate_query(question)

        # Step 1: Decompose (graceful degradation on failure)
        try:
            sub_questions = self._decompose_question(question)
            self.logger.info(f"Decomposed into {len(sub_questions)} sub-questions")
        except Exception as e:
            self.logger.error(f"Decomposition failed, using original: {e}")
            self.metrics.record_error("decomposer", str(e))
            sub_questions = [question]

        # Step 2: Retrieve
        try:
            context_chunks = self._retrieve(sub_questions)
        except Exception as e:
            self.logger.error(f"Retrieval failed: {e}")
            return {"answer": "Error retrieving information.", "error": str(e)}

        if not context_chunks:
            return {"answer": "No relevant information found.", "context_count": 0}

        # Step 3: Synthesize (returns raw context on failure)
        try:
            answer = self._synthesize(question, sub_questions, context_chunks)
        except Exception as e:
            self.logger.error(f"Synthesis failed: {e}")
            return {
                "answer": "Found relevant data but failed to generate answer.",
                "context_chunks": [c["text"][:200] for c in context_chunks],
            }

        total_ms = (time.time() - start_time) * 1000
        self.metrics.queries_processed += 1

        return {
            "answer": answer,
            "sub_questions": sub_questions,
            "sources": list(set(c["metadata"]["source_url"] for c in context_chunks)),
            "context_count": len(context_chunks),
            "latency_ms": total_ms,
        }

    def _decompose_question(self, question):
        prompt = f"""Break this question into 2-5 simpler, searchable sub-questions.
Return ONLY a JSON list of strings.

Question: \"{question}\"

Sub-questions:"""
        response = self.llm_client.chat.completions.create(
            model=self.config.llm_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.config.llm_temperature_decompose, max_tokens=500,
        )
        raw = response.choices[0].message.content.strip()
        try:
            return json.loads(raw)
        except json.JSONDecodeError:
            match = re.search(r'\[.*\]', raw, re.DOTALL)
            return json.loads(match.group()) if match else [question]

    def _retrieve(self, sub_questions, n_per_query=3):
        all_results, seen = [], set()
        for sq in sub_questions:
            emb = self.embedding_model.encode([sq]).tolist()
            results = self.collection.query(query_embeddings=emb, n_results=n_per_query,
                                            include=["documents", "metadatas", "distances"])
            for i in range(len(results["ids"][0])):
                doc_id = results["ids"][0][i]
                if doc_id not in seen:
                    seen.add(doc_id)
                    all_results.append({
                        "text": results["documents"][0][i],
                        "metadata": results["metadatas"][0][i],
                        "distance": results["distances"][0][i],
                    })
        return all_results[:self.config.max_context_chunks]

    def _synthesize(self, question, sub_questions, chunks):
        ctx = "\n\n".join(f"[Source: {c['metadata']['title']}]\n{c['text']}" for c in chunks)
        prompt = f"""Answer based ONLY on the provided context. If insufficient, say so.

Sub-questions: {json.dumps(sub_questions)}

Context:\n{ctx}

Question: {question}

Answer with citations:"""
        response = self.llm_client.chat.completions.create(
            model=self.config.llm_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.config.llm_temperature_synthesize,
            max_tokens=self.config.llm_max_tokens,
        )
        if response.usage:
            self.metrics.total_tokens_used += response.usage.total_tokens
        return response.choices[0].message.content.strip()

    # ---- HEALTH CHECK ----

    def health_check(self) -> Dict[str, Any]:
        """Health check for k8s / load balancer probes."""
        checks = {}
        try:
            checks["vector_db"] = {"status": "healthy", "count": self.collection.count()}
        except Exception as e:
            checks["vector_db"] = {"status": "unhealthy", "error": str(e)}
        try:
            self.embedding_model.encode(["test"])
            checks["embedding_model"] = {"status": "healthy"}
        except Exception as e:
            checks["embedding_model"] = {"status": "unhealthy", "error": str(e)}
        checks["metrics"] = self.metrics.to_dict()
        checks["overall"] = "healthy" if all(
            v.get("status") == "healthy" for v in checks.values() if isinstance(v, dict) and "status" in v
        ) else "degraded"
        return checks


# ============================================================
# USAGE
# ============================================================
config = AppConfig(log_level="INFO")
pipeline = ProductionRAGPipeline(config)

ingestion_results = await pipeline.ingest_urls(ARXIV_URLS + PUBMED_URLS)
for url, count in ingestion_results.items():
    print(f"  {url}: {count} chunks")

result = pipeline.query("What are the dangerous medical conditions AI might misdiagnose?")
print(f"\nAnswer: {result['answer']}")
print(f"Sources: {result.get('sources', [])}")
print(f"Latency: {result.get('latency_ms', 0):.0f}ms")
print(f"\nHealth: {json.dumps(pipeline.health_check(), indent=2)}")

### Summary of Production Hardening Applied:

| Category | Implementation |
|----------|---------------|
| **Logging** | Structured logging with levels, timestamps, component names |
| **Error Handling** | Try/catch at every stage with graceful degradation |
| **Retries** | httpx `AsyncHTTPTransport(retries=3)` — transport-level retries |
| **Connection Pooling** | httpx `Limits` for max connections and keepalive management |
| **Async I/O** | httpx `AsyncClient` with semaphore-based concurrency control |
| **Configuration** | Centralized `AppConfig` dataclass, no hardcoded values |
| **Input Validation** | Query length limits, URL allowlisting, control char sanitization |
| **Idempotency** | `upsert` instead of `add` (safe to re-run ingestion) |
| **Metrics** | `PipelineMetrics` tracks success/failures, latency, tokens, errors |
| **Health Checks** | `health_check()` for k8s/load balancer probes |
| **Observability** | Full query tracing (sub-questions, sources, latency in response) |
| **Persistent Storage** | ChromaDB with persistent directory (survives restarts) |

**Additional items for real production (not implemented but important):**
- API rate limiting (slowapi / API gateway)
- Auth (API keys, JWT)
- Caching (Redis) for repeated queries
- Async task queue (Celery) for ingestion
- Docker + k8s orchestration
- CI/CD with automated tests
- Secrets management (Vault, AWS Secrets Manager)
- Backups and disaster recovery
- Load testing

---
## Task 4: Evaluation — Measuring RAG Quality

### Evaluation Strategy

RAG systems have multiple components that can fail independently, so we need **multi-dimensional evaluation**:

#### 1. Retrieval Quality (Does the vector DB return relevant chunks?)
- **Metric**: Retrieval Precision@K
- **How**: For each query, ask an LLM judge if each retrieved chunk is relevant

#### 2. Answer Faithfulness (Is the answer grounded in the retrieved context?)
- **Metric**: Faithfulness Score (LLM-as-Judge)
- **How**: Verify every claim in the answer is supported by the context. Catches hallucinations.

#### 3. Answer Relevance (Does the answer address the question?)
- **Metric**: Relevance Score (LLM-as-Judge)
- **How**: Rate how well the answer addresses the original question (0-1)

#### 4. Answer Correctness (Is the answer factually correct?)
- **Metric**: Correctness Score against gold-standard answers
- **How**: Compare generated answer to a reference answer

#### Why this approach?
- **Retrieval metrics** isolate search quality from generation quality
- **Faithfulness** catches the #1 RAG failure mode: hallucination
- **Relevance** ensures the system answers what was asked
- **Correctness** is ground truth validation (expensive but essential)

This follows the **RAGAS framework** (Retrieval Augmented Generation Assessment).

In [ ]:
class RAGEvaluator:
    """
    Evaluates RAG pipeline quality across multiple dimensions.

    Metrics:
    1. Retrieval Precision@K - Are retrieved docs relevant?
    2. Faithfulness - Is the answer grounded in context?
    3. Answer Relevance - Does the answer address the question?
    4. Answer Correctness - Is the answer factually right?
    """

    def __init__(self, llm_client: OpenAI, model: str = "gpt-3.5-turbo"):
        self.llm_client = llm_client
        self.model = model

    def _llm_judge(self, prompt: str) -> str:
        response = self.llm_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=500,
        )
        return response.choices[0].message.content.strip()

    def evaluate_retrieval_precision(self, query: str, retrieved_chunks: List[Dict]) -> float:
        """Use LLM-as-judge to check relevance of each retrieved chunk."""
        if not retrieved_chunks:
            return 0.0
        relevant = 0
        for chunk in retrieved_chunks:
            prompt = f"""Is this passage relevant to the query?
Query: "{query}"
Passage: "{chunk['text'][:500]}"
Answer ONLY 'yes' or 'no'."""
            if "yes" in self._llm_judge(prompt).lower():
                relevant += 1
        return relevant / len(retrieved_chunks)

    def evaluate_faithfulness(self, answer: str, context_chunks: List[Dict]) -> Tuple[float, str]:
        """Check if the answer is grounded in the provided context. Catches hallucinations."""
        context_text = "\n\n".join([c["text"] for c in context_chunks])
        prompt = f"""Determine if the answer is fully supported by the context.

Context:
{context_text[:3000]}

Answer:
{answer}

Evaluate: list claims, check support, give score 0.0-1.0.
Respond in JSON: {{"claims": [{{"claim": "...", "supported": true/false}}], "score": X.X, "explanation": "..."}}"""

        result = self._llm_judge(prompt)
        try:
            parsed = json.loads(result)
            return parsed.get("score", 0.0), parsed.get("explanation", "")
        except json.JSONDecodeError:
            match = re.search(r'(\d\.\d+)', result)
            return (float(match.group(1)) if match else 0.5), result[:200]

    def evaluate_relevance(self, question: str, answer: str) -> Tuple[float, str]:
        """Rate how well the answer addresses the question."""
        prompt = f"""Rate how well this answer addresses the question.
Question: "{question}"
Answer: "{answer}"
Score 0.0-1.0. Respond in JSON: {{"score": X.X, "explanation": "..."}}"""

        result = self._llm_judge(prompt)
        try:
            parsed = json.loads(result)
            return parsed.get("score", 0.0), parsed.get("explanation", "")
        except json.JSONDecodeError:
            match = re.search(r'(\d\.\d+)', result)
            return (float(match.group(1)) if match else 0.5), result[:200]

    def evaluate_correctness(self, question: str, answer: str, reference: str) -> Tuple[float, str]:
        """Compare generated answer against a gold-standard reference."""
        prompt = f"""Compare the generated answer to the reference for correctness.
Question: "{question}"
Reference: "{reference}"
Generated: "{answer}"
Score 0.0-1.0. Consider: accuracy, completeness, absence of false claims.
Respond in JSON: {{"score": X.X, "explanation": "..."}}"""

        result = self._llm_judge(prompt)
        try:
            parsed = json.loads(result)
            return parsed.get("score", 0.0), parsed.get("explanation", "")
        except json.JSONDecodeError:
            match = re.search(r'(\d\.\d+)', result)
            return (float(match.group(1)) if match else 0.5), result[:200]

    def evaluate_full(self, question, answer, context_chunks, reference_answer=None):
        """Run all evaluation metrics on a single query-answer pair."""
        results = {}
        results["retrieval_precision"] = self.evaluate_retrieval_precision(question, context_chunks)

        faith_score, faith_exp = self.evaluate_faithfulness(answer, context_chunks)
        results["faithfulness"] = {"score": faith_score, "explanation": faith_exp}

        rel_score, rel_exp = self.evaluate_relevance(question, answer)
        results["relevance"] = {"score": rel_score, "explanation": rel_exp}

        scores = [results["retrieval_precision"], faith_score, rel_score]

        if reference_answer:
            corr_score, corr_exp = self.evaluate_correctness(question, answer, reference_answer)
            results["correctness"] = {"score": corr_score, "explanation": corr_exp}
            scores.append(corr_score)

        results["composite_score"] = sum(scores) / len(scores)
        return results


print("RAGEvaluator class defined.")

In [ ]:
# ============================================================
# Run evaluation on test cases
# ============================================================

test_cases = [
    {
        "question": "What are the risks of using AI for medical diagnosis?",
        "reference_answer": (
            "Key risks include: hallucination of medical facts, bias in training data "
            "leading to disparate outcomes, lack of explainability, automation bias, "
            "and misdiagnosis of rare conditions. Safety measures include human oversight, "
            "clinical validation, and demographic performance monitoring."
        )
    },
    {
        "question": "How do LLMs perform on medical licensing exams?",
        "reference_answer": (
            "LLMs like GPT-4 and Med-PaLM have achieved passing scores on USMLE, "
            "with some scoring above average. However, exam performance does not "
            "directly translate to clinical competence."
        )
    },
    {
        "question": "What NLP techniques are used in biomedical text mining?",
        "reference_answer": None  # No reference — evaluates retrieval, faithfulness, relevance only
    },
]

evaluator = RAGEvaluator(llm_client=OpenAI())
all_eval_results = []

for tc in test_cases:
    print(f"\n{'='*60}")
    print(f"Evaluating: {tc['question'][:60]}...")

    sub_questions = decompose_question(tc["question"])
    context_chunks = retrieve_for_subquestions(sub_questions)
    answer = synthesize_answer(tc["question"], sub_questions, context_chunks)

    eval_result = evaluator.evaluate_full(
        question=tc["question"],
        answer=answer,
        context_chunks=context_chunks,
        reference_answer=tc.get("reference_answer"),
    )
    eval_result["question"] = tc["question"]
    all_eval_results.append(eval_result)

    print(f"  Retrieval Precision: {eval_result['retrieval_precision']:.2f}")
    print(f"  Faithfulness:        {eval_result['faithfulness']['score']:.2f}")
    print(f"  Relevance:           {eval_result['relevance']['score']:.2f}")
    if "correctness" in eval_result:
        print(f"  Correctness:         {eval_result['correctness']['score']:.2f}")
    print(f"  Composite Score:     {eval_result['composite_score']:.2f}")

In [ ]:
# Summary report
print("\n" + "=" * 60)
print("EVALUATION SUMMARY REPORT")
print("=" * 60)

n = len(all_eval_results)
avg_prec = sum(r["retrieval_precision"] for r in all_eval_results) / n
avg_faith = sum(r["faithfulness"]["score"] for r in all_eval_results) / n
avg_rel = sum(r["relevance"]["score"] for r in all_eval_results) / n
avg_comp = sum(r["composite_score"] for r in all_eval_results) / n

corr_results = [r for r in all_eval_results if "correctness" in r]
avg_corr = sum(r["correctness"]["score"] for r in corr_results) / len(corr_results) if corr_results else None

print(f"\nTest cases: {n}")
print(f"  Retrieval Precision@K : {avg_prec:.2f}")
print(f"  Faithfulness          : {avg_faith:.2f}")
print(f"  Answer Relevance      : {avg_rel:.2f}")
if avg_corr is not None:
    print(f"  Answer Correctness    : {avg_corr:.2f} ({len(corr_results)} with references)")
print(f"  Composite Score       : {avg_comp:.2f}")

print(f"\n{'='*60}")
print("INTERPRETATION:")
print("  0.8-1.0 : Excellent — production ready")
print("  0.6-0.8 : Good — minor improvements needed")
print("  0.4-0.6 : Fair — significant improvements needed")
print("  0.0-0.4 : Poor — fundamental issues")
print(f"{'='*60}")

### Evaluation Design Rationale

**Why LLM-as-Judge?**
- BLEU/ROUGE measure surface text overlap — poor proxy for RAG answer quality
- LLM-as-Judge correlates much better with human judgment
- Cost-effective compared to human evaluation at scale

**Why these 4 metrics?**
1. **Retrieval Precision** → Isolates vector DB quality from LLM quality
2. **Faithfulness** → The #1 RAG failure mode is hallucination. This directly measures it.
3. **Relevance** → Ensures the system answers what was asked
4. **Correctness** → Ground truth validation where available

**Tradeoffs acknowledged:**
- LLM-as-Judge has its own biases (prefers longer answers, own style)
- Gold-standard test sets are expensive but essential
- In production: add human eval on sample + automated regression tests
- Also track: latency, cost/query, user satisfaction (thumbs up/down)